In [1]:
print('Hello World')

Hello World


In [2]:
import argparse, torch, random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [3]:

def load_model(model_id: str):
    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb,
        device_map="auto",
        trust_remote_code=False,
    )
    model.eval()
    return tok, model

In [4]:
def generate(tok, model, prompt, max_new_tokens=128, temperature=0.7):
    dev = model.device
    inputs = tok(prompt, return_tensors="pt").to(dev)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tok.eos_token_id,
        )
    return tok.decode(out[0], skip_special_tokens=True)

In [6]:
def converse(prompt):
    model = "unsloth/mistral-7b-instruct-v0.3"

    tok, model = load_model(model)
    print(generate(tok, model, prompt))

converse('Hello, is anyone there?')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Hello, is anyone there?

Does anyone read this stuff?

I'm thinking of writing a little article about the history of this forum. I have to admit, I've never posted here before, although I have read the forum before. But to be honest, I've never had much of a reason to, as other forums seemed to cover the same topics (i.e. the history of the town and the surrounding area). However, I noticed that this forum seems to be pretty inactive these days. I'm assuming this is because of the rise of social media and the internet in general. I'm not here
